In [1]:
import os, gc
import torch

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [3]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer, MultiNormalizer
from pytorch_forecasting.metrics import RMSE, MultiLoss

In [5]:
from dataclasses import dataclass

@dataclass
class args:
    result_folder = '../results/original/AGE1829/'
    figPath = os.path.join(result_folder, 'figures')
    checkpoint_folder = os.path.join(result_folder, 'checkpoints')
    input_filePath = '../2022_May_age_groups/Total.csv'
    configPath = '../configurations/age_groups.json'

    # Path/URL of the checkpoint from which training is resumed
    ckpt_model_path = None # os.path.join(checkpoint_folder, 'latest-epoch=0.ckpt')
    
    # set this to false when submitting batch script, otherwise it prints a lot of lines
    show_progress_bar = False
    static_feature_index = 2

In [6]:
total_data = pd.read_csv(args.input_filePath)
print(total_data.shape)
total_data.head()

(2541878, 14)


,FIPS,UNDER5,AGE517,AGE1829,AGE3039,AGE4049,AGE5064,AGE6574,AGE75PLUS,Date,VaccinationFull,Cases,TimeFromStart,SinWeekly
0,1001,0.0596,0.1725,0.1468,0.1315,0.1321,0.1965,0.0922,0.0689,2020-12-13,0.0,0.0,288,-0.7818
1,1001,0.0596,0.1725,0.1468,0.1315,0.1321,0.1965,0.0922,0.0689,2020-12-14,0.0,96.0,289,0.0000
2,1001,0.0596,0.1725,0.1468,0.1315,0.1321,0.1965,0.0922,0.0689,2020-12-15,0.0,97.0,290,0.7818
3,1001,0.0596,0.1725,0.1468,0.1315,0.1321,0.1965,0.0922,0.0689,2020-12-16,0.0,84.0,291,0.9749
4,1001,0.0596,0.1725,0.1468,0.1315,0.1321,0.1965,0.0922,0.0689,2020-12-17,0.0,60.0,292,0.4339


In [7]:
import json
import sys
sys.path.append( '..' )
from Class.Parameters import Parameters
from script.utils import *

with open(args.configPath, 'r') as input_file:
  config = json.load(input_file)

parameters = Parameters(config, **config)

print(f'Static features {parameters.data.static_features}. \
  Choosing feature {parameters.data.static_features[args.static_feature_index]}.')
# here we are going to train by one static feature at a time
parameters.data.static_features = [parameters.data.static_features[args.static_feature_index]]

# %%
targets = parameters.data.targets
time_idx = parameters.data.time_idx
tft_params = parameters.model_parameters

max_prediction_length = tft_params.target_sequence_length
max_encoder_length = tft_params.input_sequence_length

Static features ['UNDER5', 'AGE517', 'AGE1829', 'AGE3039', 'AGE4049', 'AGE5064', 'AGE6574', 'AGE75PLUS'].   Choosing feature AGE1829.


In [8]:
import random

def seed_torch(seed=7):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    pl.seed_everything(seed)

seed_torch(tft_params.seed)

[rank: 0] Global seed set to 7


In [9]:
total_data['Date'] = pd.to_datetime(total_data['Date'].values) 
total_data['FIPS'] = total_data['FIPS'].astype(str)
print(f"There are {total_data['FIPS'].nunique()} unique counties in the dataset.")

There are 3142 unique counties in the dataset.


In [10]:
scale_data

<function script.utils.scale_data(train_data: pandas.core.frame.DataFrame, validation_data: pandas.core.frame.DataFrame, test_data: pandas.core.frame.DataFrame, parameters: Class.Parameters.Parameters)>